# **Assignment No.5**
### By BTAIMLA71

# Import Libraries

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

# Importing Dataset

In [3]:
# Step 2: Loading the Breast Cancer dataset

data = pd.read_csv(
    r"C:\PCCOE Notes\RS\RS Practical 5\Womens Clothing E-Commerce Reviews.csv"
)
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [6]:
# Aggregate to get the mean rating for each (Age, Clothing ID) pair to ensure uniqueness
data = data.groupby(["Age", "Clothing ID"], as_index=False).agg({"Rating": "mean"})

In [7]:
# Convert data to a user-item matrix with 'Age' as user ID
user_item_matrix = data.pivot(
    index="Age", columns="Clothing ID", values="Rating"
).fillna(0)

In [12]:
user_item_matrix.head()

Clothing ID,0,1,2,3,4,5,6,7,8,9,...,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205
Age,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
user_item_matrix.describe()
user_item_matrix.isna()
user_item_matrix.isnull()

Clothing ID,0,1,2,3,4,5,6,7,8,9,...,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205
Age,,,,,,,,,,,,,,,,,,,,,
18,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
92,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
# Convert the user-item matrix to a sparse matrix
user_item_sparse_matrix = csr_matrix(user_item_matrix.values)

In [15]:
# Matrix factorization using Singular Value Decomposition (SVD)
# Choosing k (number of latent features) as 2 for simplicity; adjust for better accuracy
U, sigma, Vt = svds(user_item_sparse_matrix, k=2)
sigma = np.diag(sigma)

In [16]:
# Reconstruct the prediction matrix
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)

In [25]:
predicted_ratings_df.head()

Clothing ID,0,1,2,3,4,5,6,7,8,9,...,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205
Age,,,,,,,,,,,,,,,,,,,,,
18,0.013939,0.000369,0.014885,-0.020908,0.018606,-0.033786,-0.033786,-0.033786,-0.000075,-0.004753,...,0.003921,0.050011,0.020483,0.003130,0.009401,-0.024848,0.070560,0.024725,-0.030408,0.009689
19,0.045082,0.040023,0.043046,-0.024800,0.053807,-0.050779,-0.050779,-0.050779,0.064333,0.005233,...,0.023797,0.143969,0.060342,0.020324,0.026986,-0.022901,0.273149,0.187046,-0.045701,0.048899
20,0.108626,0.050540,0.109745,-0.110375,0.137182,-0.191499,-0.191499,-0.191499,0.078683,-0.011745,...,0.044199,0.367919,0.152376,0.036913,0.069066,-0.123107,0.605045,0.324124,-0.172349,0.097067
21,0.117636,0.055026,0.118809,-0.119205,0.148512,-0.206938,-0.206938,-0.206938,0.085700,-0.012563,...,0.047949,0.398301,0.164970,0.040052,0.074769,-0.132882,0.655570,0.351821,-0.186245,0.105251
22,0.131110,0.098447,0.127544,-0.091920,0.159430,-0.174719,-0.174719,-0.174719,0.157246,0.005695,...,0.064068,0.426919,0.178219,0.054392,0.080064,-0.093167,0.773612,0.494478,-0.157247,0.134094


In [17]:
# Define recommendation function
def recommend_items(user_age, num_recommendations=3):
    if user_age not in predicted_ratings_df.index:
        raise ValueError("User age not found in the dataset.")
    user_predictions = predicted_ratings_df.loc[user_age].sort_values(ascending=False)
    recommended_items = user_predictions.head(num_recommendations).index.tolist()
    return recommended_items

In [24]:
# Example: Recommendations for user with age 34
try:
    recommendations = recommend_items(user_age=34)
    print("Recommendations for user aged 34:", recommendations)
except ValueError as e:
    print(e)

Recommendations for user aged 34: [984, 1083, 834]


In [19]:
# Evaluate the model using Root Mean Squared Error (RMSE)
# Split data into train and test, then create user-item matrices for both
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_matrix = train_data.pivot(
    index="Age", columns="Clothing ID", values="Rating"
).fillna(0)
test_matrix = test_data.pivot(
    index="Age", columns="Clothing ID", values="Rating"
).fillna(0)

In [20]:
# Calculate RMSE only for ratings that exist in both matrices
common_indices = test_matrix.index.intersection(predicted_ratings_df.index)
common_columns = test_matrix.columns.intersection(predicted_ratings_df.columns)
test_matrix = test_matrix.loc[common_indices, common_columns]
predicted_matrix = predicted_ratings_df.loc[common_indices, common_columns]

In [21]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test_matrix.values, predicted_matrix.values))
print("RMSE of the model:", rmse)

RMSE of the model: 1.3985110616255154


In [ ]:
"""
1. Introduction to Recommendation Systems

Recommendation systems are essential tools in e-commerce platforms, designed to enhance user
experience by providing personalized suggestions. These systems analyze user behavior and
preferences to recommend products that are most likely to interest individual users. They can
significantly increase user engagement, boost sales, and improve customer satisfaction.
Types of Recommendation Systems:
- Collaborative Filtering: Based on the idea that users with similar tastes will like similar items.
This method uses historical interactions among users and items.
- Content-Based Filtering: Recommends items similar to those a user has previously liked, based
on item attributes (e.g., features, descriptions).
- Hybrid Methods: Combine collaborative and content-based filtering to leverage the strengths of
both approaches, providing more accurate and diverse recommendations.
2. Matrix Factorization
Matrix factorization is a powerful technique used in collaborative filtering. It works by
decomposing the user-item interaction matrix into lower-dimensional matrices that represent
latent features of users and items. These latent features help capture hidden patterns in user
preferences.
Key Concepts:
- User-Item Interaction Matrix: This matrix (often sparse) represents interactions (e.g., ratings,
purchases) between users and items. Rows correspond to users, columns correspond to items,
and entries represent interactions.
\[ R = \begin{bmatrix}
r_{11} & r_{12} & \ldots & r_{1n} \\
r_{21} & r_{22} & \ldots & r_{2n} \\

\vdots & \vdots & \ddots & \vdots \\
r_{m1} & r_{m2} & \ldots & r_{mn}
\end{bmatrix}
\]
- Latent Factors: These are hidden features that explain observed ratings. For example, in a
movie recommendation system, latent factors could represent genres, while in e-commerce, they
could represent product categories or user preferences.
- Decomposition: The interaction matrix \( R \) can be approximated by the product of two
lower-dimensional matrices:
\[
R \approx U \times V^T
\]
where \( U \) is the user feature matrix and \( V \) is the item feature matrix.

3. Popular Matrix Factorization Techniques
- Singular Value Decomposition (SVD): A well-known technique that reduces dimensionality by
factorizing the user-item matrix into three matrices: user features, singular values, and item
features. SVD captures the most significant patterns in the data.
- Alternating Least Squares (ALS): This method iteratively solves for user and item matrices by
alternately fixing one and optimizing the other. ALS is particularly useful for large datasets and
is effective in implicit feedback scenarios.
- Non-negative Matrix Factorization (NMF): Decomposes the matrix into non-negative factors,
making the results easier to interpret. This is beneficial in scenarios where negative values are
not meaningful, such as counts or ratings.

4. Model Training and Evaluation
Training a matrix factorization model involves the following steps:
1. Loss Function: The model's performance is typically measured using a loss function, such as
Mean Squared Error (MSE):
\[
\text{Loss} = \sum_{(i,j) \in K} (r_{ij} - \hat{r}_{ij})^2
\]
where \( r_{ij} \) is the actual rating and \( \hat{r}_{ij} \) is the predicted rating. The sum runs
over all known interactions \( K \).
2. Regularization: To avoid overfitting, regularization techniques are applied to the loss function,
penalizing large weights in the latent factor matrices.

3. Gradient Descent: This optimization technique is often used to minimize the loss function,
iteratively adjusting the latent factors based on the gradients.

5. Recommendations and User Engagement
Once trained, the model can predict missing ratings for items not yet interacted with by users.
Recommendations can be generated by:
- Predicting Scores: Calculate the predicted interaction scores for each user-item pair.
- Top-N Recommendations: For each user, recommend the top-N items with the highest
predicted scores.
"""